## Solutions

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv("athlete_events.csv")

1) Look at 10 rows of the dataset (top or bottom).

In [ ]:
df.head(10)

2) Look at the distribution of values (value counts of different values) in the column `'Medal'`.

Notice how the function used sorts values in descending order - it will be useful later on.

In [ ]:
df["Medal"].value_counts()

3) Check the distribution of `NaNs` across columns. What does the presence of `Nan` mean in each column?

In [ ]:
df.isna().sum()

4. Calculate participation according to year in sport.

In [ ]:
df.groupby(["Sport", "Year"])["ID"].count()

5. Calculate percentage athletes who won gold medal grouping by sport.

In [ ]:
df[df["Medal"] == "Gold"].groupby(["Sport", "Year"])["ID"].count() / df[
    df["Medal"] != "Gold"
].groupby(["Sport", "Year"])["ID"].count()

6) Select only the rows describing gold medalists and store them into `gold_df` variable.

In [ ]:
gold_df = df[df["Medal"] == "Gold"].copy()
gold_df

7) Calculate the average age of gold medalist.

In [ ]:
gold_df["Age"].mean()

8) Show top 10 countries with respect to the number of gold medals won.

In [ ]:
gold_df.groupby("Team").count().sort_values(by="Medal", ascending=False)["Medal"]

9) Calculate the number of golden medals of Belgium.

In [ ]:
gold_df[gold_df["Team"] == "Belgium"]["Medal"].count()

10) Calculate number of Polish gold medals by year.

In [ ]:
gold_df_poland = gold_df[gold_df["Team"] == "Poland"]

In [ ]:
gold_df_poland.groupby("Year")["ID"].count()

11) Visualise it using function `plot()`.

In [ ]:
gold_df_poland = gold_df_poland.groupby("Year")["ID"].count().to_frame()

In [ ]:
gold_df_poland.plot()

12) Select all of the rows describing Polish gold medals in 1972 to analyze the anomaly.

In [ ]:
poland = gold_df.query("Year == 1972 and Team == 'Poland'")
poland

13) Calculate weight to height ratio for every athlete.

In [ ]:
df["Weight"] / df["Height"]

14) Create a new column with the average height for a given sex and age of athletes (not only gold medalists). Call it `'typical_height'`.

In [ ]:
def height_calculator(group):
    group["typical_height"] = group.Height.sum() / len(group)
    return group


df = df.groupby(["Sex", "Age"]).apply(height_calculator)
df

15) Create a new column `'height_diff'` with the deviation of height from the typical value in the segment.

In [ ]:
df["height_diff"] = df["Height"] - df["typical_height"]
df

16) Drop the rows in the dataset where the newly calculated value is `NaN`. How many rows have been dropped?

Hint: use `dropna()` function on the dataset with `subset=['height_diff]` as the only argument.

In [ ]:
print(df.height_diff.isna().sum())
df.dropna(inplace=True, subset=["height_diff"])

17) Plot the histogram of this newly calculated column.

Hint: use `seaborn.distplot` function, passing the appropriate column as the only argument.

In [ ]:
import seaborn as sns

sns.histplot(df["height_diff"])

18) Verify whether the majority of gold medalists were higher than the typical height of their segment.

In [ ]:
(df.query("height_diff > 0 and Medal == 'Gold'").shape[0] / df.shape[0]) > 0.5

19) Calculate the actual percentage of gold medalists higher than the `'typical height'`.

In [ ]:
df.query("height_diff > 0 and Medal == 'Gold'").shape[0] / df.shape[0]

20) Calculate mean age and standard deviation of gold medalists. Compare it with the youngest gold medalist.

In [ ]:
gold_df.describe()["Age"]

21) Add new columns `first_gold` and `last_gold` meaning the first and last years of obtaining a gold medals by an athlete.

In [ ]:
def min_year(group):
    group["first_gold"] = group.Year.min()
    return group


def max_year(group):
    group["last_gold"] = group.Year.max()
    return group


gold_df = gold_df.groupby(["Name"]).apply(min_year)
gold_df = gold_df.groupby(["Name"]).apply(max_year)
gold_df[["Year", "first_gold", "last_gold"]]

22) Add a new column `'gold_span'` with the span in years of achieved gold medals. What is the maximal value of this span?

In [ ]:
gold_df["gold_span"] = list(zip(gold_df.first_gold, gold_df.last_gold))
gold_df["gold_span_difference"] = gold_df.last_gold - gold_df.first_gold

In [ ]:
gold_df.sort_values(by="gold_span_difference", ascending=False)

In [ ]:
gold_df.gold_span_difference.max()

23) Select the rows describing the history of this athlete/these athletes that reached maximal span.

In [ ]:
top_athlete = gold_df[
    gold_df["gold_span_difference"] == int(gold_df.gold_span_difference.max())
]

In [ ]:
top_athlete

24. Find top performers in each Olympics (athletes who won the most medals).

In [ ]:
medalists = df.dropna(inplace=False, subset=["Medal"])
medalists.groupby(["Name"]).count()["ID"].sort_values(ascending=False)

25) Who obtained the highest number of silver medals without ever getting a gold one and how many of them?

In [ ]:
medalists = df.dropna(inplace=False, subset=["Medal"])

gold_medallists_ids = medalists[medalists["Medal"] == "Gold"].ID.unique()

medalists_no_gold = medalists[~df["ID"].isin(gold_medallists_ids)]

In [ ]:
medalists_no_gold_silver = medalists_no_gold[medalists_no_gold["Medal"] == "Silver"]

In [ ]:
medalists_no_gold_silver.groupby(["Name"]).count()["ID"].sort_values(ascending=False)

26. Create a new DataFrame called "gold_medals_one_hot", and fill it with values from the "Medal" column encoded using one-hot method. Explanation and example of one-hot encoding can be found [here](https://en.wikipedia.org/wiki/One-hot)

In [ ]:
gold_medals_one_hot = pd.get_dummies(df["Medal"])

In [ ]:
gold_medals_one_hot["No_medal"] = np.zeros(len(one_hot))

In [ ]:
gold_medals_one_hot.loc[gold_medals_one_hot.sum(axis=1) == 0, "No_medal"] = 1

In [ ]:
gold_medals_one_hot